# README
This cookbook was created by Joe Constantino on June 25, 2024 to test an NLQ-to-VDS query pipeline for answering ad-hoc questions over Tableau Published Data Sources

This cookbook can be configured to query a Tableau Published data source in natural language using the Headless BI REST endpoint. It uses langchain libraries and the langchain development framework to generate a headless BI request payload from a natural language input query. It then executes a chain to generate an answer to the user's input query.

To get started with this notebook, make sure you've added your environment variables to your .env file. The environment variables are set up to consume a PAT token associated to a specific Tableau site. More info on authenticating to VizQL Data Service can be found here: https://help.tableau.com/v0.0/api/vizql-data-service/en-us/docs/vds_setup.html

We recommend using gpt-4o or gpt-4o-mini for the VDS agent chain.

In [4]:
#import all required libraries
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import pandas as pd
import os
import json
import requests
import re

from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool

# loading environment variables
env_vars = os.environ

In [6]:
#read in the datasource metadata of the datasource that is defined in your environment variables
def read():
    url = env_vars['READ_METADATA']
    payload = json.dumps({
        "connection": {
            "tableauServerName": env_vars['TABLEAU_DOMAIN'],
            "siteId": env_vars['SITE_NAME'],
            "datasource": env_vars['DATA_SOURCE']
        },
    })

    headers = {
    'Credential-Key': env_vars['PAT_NAME'],
    'Credential-value': env_vars['PAT_SECRET'],
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Print the response data
        #print("Response Data:")
        #print(response.json())
        data = response.json()['data']
        # Create a pandas DataFrame from the JSON data
        #df = pd.DataFrame(data)

        # Display the first few rows of the DataFrame
        #print("Table view of data from the public REST API:")
        #print(df.head())
        #print(df.all())
        return data
        #display(df.head())
    else:
        print("Failed to fetch data from the API. Status code:", response.status_code)
        print(response.text)
datasource_metadata = read()

In [11]:
#define the headless BI query template
def send(query):
    url = env_vars['VDS_URL']
    payload = json.dumps({
        "connection": {
            "tableauServerName": env_vars['TABLEAU_DOMAIN'],
            "siteId": env_vars['SITE_NAME'],
            "datasource": env_vars['DATA_SOURCE']
        },
        "query": query
    })

    headers = {
    'Credential-Key': env_vars['PAT_NAME'],
    'Credential-value': env_vars['PAT_SECRET'],
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Print the response data
        #print("Response Data:")
        #print(response.json())
        data = response.json()['data']
        # Create a pandas DataFrame from the JSON data
        df = pd.DataFrame(data)

        # Display the first few rows of the DataFrame
        #print("Table view of data from the public REST API:")
        #print(df.head())
        #print(df.all())
        return df
        #display(df.head())
    else:
        print("Failed to fetch data from the API. Status code:", response.status_code)
        print(response.text)

In [23]:
# Specify the path to the system prompt
path = './prompts/nlq_to_vds.py'

# read in system prompt
with open(path, 'r') as json_file:
    headless_bi_prompt_string = json_file.read()

#remove "prompt = " prefix from the string
stripped_headless_bi_prompt_string = headless_bi_prompt_string.replace('prompt = ', '', 1).strip()

#convert headless bi prompt string to a json object
headless_bi_prompt = json.loads(stripped_headless_bi_prompt_string)

# add the datasource metadata of the connected datasource to the system prompt
headless_bi_prompt['data_model'] = datasource_metadata

#convert prompt back to a string to pass in LLM request
ready_prompt = json.dumps(headless_bi_prompt)
print(ready_prompt)

{"instructions": "You are an expert at writing JSON payloads for VDS queries. \nVDS queries are passed in as a JSON body that describes the query, and the return is passed back as an array of JSON objects. The VDS query is a JSON object. It contains two fundamental components.\n1. columns [required] - an array of columns that define the desired output of the query\n2. filters [optional] - an array of filters to apply to the query. They can include fields that are not in the columns array.\nYour task is to retrieve data relevant to a natural language query. A pandas AI Agent will be used to transform and analyze the data that your VDS query returns. Don't try to do too much with the json query. You will be successful if you bring back all the data that is required to answer the question, even if additional transformation and actions are needed. Query as much data as might be useful; it's ok if you pull in superfluous columns, but only pull in columns based on what is available in the da

In [13]:
#define function for getting user queries
def get_utterance():
  query = input('what would you like to know about your data? reply stop if you are done.')
  return query


In [16]:
active_utterance = get_utterance()

while (active_utterance != 'stop'):
  llm = ChatOpenAI(model=env_vars['VDS_AGENT_MODEL'])
  active_prompt_template = ChatPromptTemplate.from_messages([
      SystemMessage(content=headless_bi_prompt_string),
      ("user", "{utterance}")])
  prompt_value = active_prompt_template.invoke(
      {
          "utterance": active_utterance
      }
  )
  output_parser = StrOutputParser()
  chain = active_prompt_template | llm | output_parser
  output = chain.invoke(active_utterance)

  #print reasoning
  print(output.split('JSON_payload')[0])

  #parse LLM output and query headless BI
  parsed_output = output.split('JSON_payload')[1]
  match = re.search(r'{.*}', parsed_output, re.DOTALL)
  if match:
    json_string = match.group(0)
    payload = json.loads(json_string)
  #print(payload)
  df = send(payload)
  agent = create_pandas_dataframe_agent(
      ChatOpenAI(temperature=0, model=env_vars['PANDAS_AGENT_MODEL']),
      df,
      verbose=True,
      agent_type=AgentType.OPENAI_FUNCTIONS,
      allow_dangerous_code=True
  )
  instruct_header = "answer the following query directly by executing the necessary python code. Give a succinct explanation of the steps you took and how you know the answer is correct: "
  agent.invoke(instruct_header + active_utterance)
  active_utterance = get_utterance()

what would you like to know about your data? reply stop if you are done. what are sales per region? 


Reasoning: The user is asking for sales data categorized by region. Therefore, we need to include the "Region" column and aggregate the "Sales" column using the SUM function to get the total sales per region.




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "sales_per_region = df.set_index('Region')['SUM(Sales)']\nsales_per_region"}`


Region
West       739813.61
East       691828.17
Central    503170.67
South      391721.91
Name: SUM(Sales), dtype: float64To find the sales per region, I executed the following steps:

1. **Set the Index**: I set the 'Region' column as the index of the dataframe using `set_index('Region')`. This allows us to easily reference sales data by region.
2. **Select Sales Data**: I accessed the 'SUM(Sales)' column to get the sales figures corresponding to each region.
3. **Result**: The result is a Series with regions as the index and their corresponding sales values.

The output shows the sales figures for each region, con

what would you like to know about your data? reply stop if you are done. which products are the most discounted? 


Reasoning: To identify the products that are most discounted, we will need to retrieve data on product names and their respective discount amounts. The query will include the "Product" column and the "Discount" column, applying a sorting function to display products in descending order based on their discount amount.


Failed to fetch data from the API. Status code: 400
{"errorCode":"400002","message":"Column 'Product' does not exist","datetime":"2024-08-25T14:24:55.573619Z","details":{"errorCode":"500104","message":"Base column [DSInternalName].[Product] does not exist.","datetime":"2024-08-25T14:24:55.489511Z","details":null}}


ValueError: Expected pandas DataFrame, got <class 'NoneType'>

In [27]:
#test nlq_to_vds chain
test_utterance = "which products tend to have the largest discounts?"
llm = ChatOpenAI(model=env_vars['VDS_AGENT_MODEL'])
active_prompt_template = ChatPromptTemplate.from_messages([
  SystemMessage(content=headless_bi_prompt_string),
  ("user", "{utterance}")])
prompt_value = active_prompt_template.invoke(
  {
      "utterance": test_utterance
  }
)
output_parser = StrOutputParser()
chain = active_prompt_template | llm | output_parser
output = chain.invoke(test_utterance)

#print reasoning
print(output.split('JSON_payload')[0])

#parse LLM output and print generated json
parsed_output = output.split('JSON_payload')[1]
match = re.search(r'{.*}', parsed_output, re.DOTALL)
if match:
    json_string = match.group(0)
json_payload = json.loads(json_string)
print(json_payload)

Reasoning: To determine which products tend to have the largest discounts, I will query the relevant columns that represent the products and their discount values. I will include the product name and the discount amount, and I will also sort the results based on the discount amount in descending order to identify the products with the largest discounts.


{'columns': [{'columnName': 'Product'}, {'columnName': 'Discount Amount', 'function': 'MAX'}], 'filters': []}
